In [1]:
import pulp
print(pulp.listSolvers(onlyAvailable=True))

['GLPK_CMD']


In [2]:
a = [1 ,2 ]
a += [3, 4]
a

[1, 2, 3, 4]

In [3]:
print(2)

2


In [4]:
a = [np.array([[0, 0]]), np.array([[1, 0]]), np.array([[0, 0]])]

NameError: name 'np' is not defined

In [ ]:
a[2]

In [ ]:
np.arctan2(1,1)/np.pi

In [ ]:
import numpy as np

# 创建一个NumPy数组
arr = np.array([425., 200.])

# 将NumPy数组转换为Python列表
arr_list = arr.tolist()

print(arr_list)

In [ ]:
import numpy as np

def mid_angle(theta1, theta2):
    # Ensure the angles are between -pi and pi
    theta1 = np.arctan2(np.sin(theta1), np.cos(theta1))
    theta2 = np.arctan2(np.sin(theta2), np.cos(theta2))

    # Compute the mid angle
    sin_avg = (np.sin(theta1) + np.sin(theta2)) / 2
    cos_avg = (np.cos(theta1) + np.cos(theta2)) / 2
    theta_mid = np.arctan2(sin_avg, cos_avg)

    return theta_mid

# Example usage
theta1 = 0.25
theta2 = -0.75

theta_mid = mid_angle(theta1, theta2)
print(theta_mid)


-0.24999999999999997


In [ ]:
"""
人工势场寻路算法实现
改进人工势场，解决不可达问题，仍存在局部最小点问题
"""

from Original_APF import APF, Vector2d
import matplotlib.pyplot as plt
import math
from matplotlib.patches import Circle
import random


def check_vec_angle(v1: Vector2d, v2: Vector2d):
    v1_v2 = v1.deltaX * v2.deltaX + v1.deltaY * v2.deltaY
    angle = math.acos(v1_v2 / (v1.length * v2.length)) * 180 / math.pi
    return angle


class APF_Improved(APF):
    def __init__(self, start: (), goal: (), obstacles: [], k_att: float, k_rep: float, rr: float,
                 step_size: float, max_iters: int, goal_threshold: float, is_plot=False):
        self.start = Vector2d(start[0], start[1])
        self.current_pos = Vector2d(start[0], start[1])
        self.goal = Vector2d(goal[0], goal[1])
        self.obstacles = [Vector2d(OB[0], OB[1]) for OB in obstacles]
        self.k_att = k_att
        self.k_rep = k_rep
        self.rr = rr  # 斥力作用范围
        self.step_size = step_size
        self.max_iters = max_iters
        self.iters = 0
        self.goal_threashold = goal_threshold
        self.path = list()
        self.is_path_plan_success = False
        self.is_plot = is_plot
        self.delta_t = 0.01

    def repulsion(self):
        """
        斥力计算, 改进斥力函数, 解决不可达问题
        :return: 斥力大小
        """
        rep = Vector2d(0, 0)  # 所有障碍物总斥力
        for obstacle in self.obstacles:
            # obstacle = Vector2d(0, 0)
            obs_to_rob = self.current_pos - obstacle
            rob_to_goal = self.goal - self.current_pos
            if (obs_to_rob.length > self.rr):  # 超出障碍物斥力影响范围
                pass
            else:
                rep_1 = Vector2d(obs_to_rob.direction[0], obs_to_rob.direction[1]) * self.k_rep * (
                        1.0 / obs_to_rob.length - 1.0 / self.rr) / (obs_to_rob.length ** 2) * (rob_to_goal.length ** 2)
                rep_2 = Vector2d(rob_to_goal.direction[0], rob_to_goal.direction[1]) * self.k_rep * ((1.0 / obs_to_rob.length - 1.0 / self.rr) ** 2) * rob_to_goal.length
                rep +=(rep_1+rep_2)
        return rep


if __name__ == '__main__':
    # 相关参数设置
    k_att, k_rep = 1.0, 0.8
    rr = 3
    step_size, max_iters, goal_threashold = .2, 500, .2  # 步长0.5寻路1000次用时4.37s, 步长0.1寻路1000次用时21s
    step_size_ = 2

    # 设置、绘制起点终点
    start, goal = (0, 0), (15, 15)
    is_plot = True
    if is_plot:
        fig = plt.figure(figsize=(7, 7))
        subplot = fig.add_subplot(111)
        subplot.set_xlabel('X-distance: m')
        subplot.set_ylabel('Y-distance: m')
        subplot.plot(start[0], start[1], '*r')
        subplot.plot(goal[0], goal[1], '*r')
    # 障碍物设置及绘制
    obs = [[1, 4], [2, 4], [3, 3], [6, 1], [6, 7], [10, 6], [11, 12], [14, 14]]
    print('obstacles: {0}'.format(obs))
    for i in range(0):
        obs.append([random.uniform(2, goal[1] - 1), random.uniform(2, goal[1] - 1)])

    if is_plot:
        for OB in obs:
            circle = Circle(xy=(OB[0], OB[1]), radius=rr, alpha=0.3)
            subplot.add_patch(circle)
            subplot.plot(OB[0], OB[1], 'xk')
    # t1 = time.time()
    # for i in range(1000):

    # path plan
    if is_plot:
        apf = APF_Improved(start, goal, obs, k_att, k_rep, rr, step_size, max_iters, goal_threashold, is_plot)
    else:
        apf = APF_Improved(start, goal, obs, k_att, k_rep, rr, step_size, max_iters, goal_threashold, is_plot)
    apf.path_plan()
    if apf.is_path_plan_success:
        path = apf.path
        path_ = []
        i = int(step_size_ / step_size)
        while (i < len(path)):
            path_.append(path[i])
            i += int(step_size_ / step_size)

        if path_[-1] != path[-1]:  # 添加最后一个点
            path_.append(path[-1])
        print('planed path points:{}'.format(path_))
        print('path plan success')
        if is_plot:
            px, py = [K[0] for K in path_], [K[1] for K in path_]  # 路径点x坐标列表, y坐标列表
            subplot.plot(px, py, '^k')
            plt.show()
    else:
        print('path plan failed')
    # t2 = time.time()
    # print('寻路1000次所用时间:{}, 寻路1次所用时间:{}'.format(t2-t1, (t2-t1)/1000))



ModuleNotFoundError: No module named 'Original_APF'

In [ ]:
a= np.array([[502.5       , 200.        ],
       [202.5       , 373.20508076],
       [202.5       ,  26.79491924]])
c = tuple(a[1])
c

(202.5, 373.20508076)

In [ ]:
for i in range(3):
    print(i)

0
1
2


In [7]:
import numpy as np
a = np.array([300.5, 200. ])
a[0]

300.5

In [9]:
import numpy as np

def contract_net_protocol(cost_dict):
    # 初始化
    targets = list(cost_dict.keys())
    ugvs = list(cost_dict[targets[0]].keys())
    n_targets = len(targets)
    n_ugvs = len(ugvs)

    assignments = {t: [] for t in targets}
    remaining_ugvs = set(ugvs)

    # 确保每个目标至少分配到3个点
    for t in targets:
        bids = [(u, cost_dict[t][u]) for u in remaining_ugvs]
        bids.sort(key=lambda x: x[1])  # 按成本排序
        for u, _ in bids[:3]:  # 选择前3个成本最低的UGV
            assignments[t].append(u)
            remaining_ugvs.remove(u)
        if len(assignments[t]) < 3:
            raise ValueError(f"无法满足目标 {t} 至少分配到3个点的约束条件")

    # 分配剩余的UGV，确保每个目标不超过5个点
    while remaining_ugvs:
        bids = []
        for t in targets:
            for u in remaining_ugvs:
                bids.append((u, t, cost_dict[t][u]))
        bids.sort(key=lambda x: x[2])  # 按成本排序
        for u, t, _ in bids:
            if len(assignments[t]) < 5:
                assignments[t].append(u)
                remaining_ugvs.remove(u)
                break

    # 确保每个目标分配到的点不超过5个
    for t in targets:
        if len(assignments[t]) > 5:
            raise ValueError(f"目标 {t} 分配的点超过5个")

    # 转换为分配列表
    result = [assignments[t] for t in targets]

    return result
# 示例使用
cost_dict = {
    "Target_0": {0: 2786.531487222121, 1: 2717.141293354679, 7: 2665.6269656421377, 2: 2758.712848725679, 6: 209.80176109551746, 3: 191.19505225805992, 4: 2546.4535064771144, 8: 211.79407586165013, 5: 188.71857182411463}, 
    "Target_1": {0: 693.8583480919154, 1: 697.2566760652994, 7: 748.83756852169, 2: 681.4751559151266, 6: 3537.702347411496, 3: 3321.0435791638993, 4: 881.2560596238144, 8: 3594.6688509045384, 5: 3258.2662164458225},
    "Target_2": {0: 2404.383627400785, 1: 2559.1189681649075, 7: 2431.784308832652, 2: 2439.4056679026744, 6: 1628.6516195994427, 3: 1835.9578257827973, 4: 2482.683700618737, 8: 1897.0929139596337, 5: 1556.4379882833684}
}

result = contract_net_protocol(cost_dict)
print(result)

[[5, 3, 6], [2, 0, 1], [8, 7, 4]]
